<a href="https://colab.research.google.com/github/AnshChoudhary/DeepSeek-VLM-Chat/blob/main/DeepSeek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio -q
import gradio as gr

In [2]:
!git clone https://github.com/deepseek-ai/DeepSeek-VL
%cd DeepSeek-VL

!pip install -e .

Cloning into 'DeepSeek-VL'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 119 (delta 28), reused 21 (delta 10), pack-reused 60
Receiving objects: 100% (119/119), 12.47 MiB | 11.68 MiB/s, done.
Resolving deltas: 100% (37/37), done.
/content/DeepSeek-VL
Obtaining file:///content/DeepSeek-VL
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 7.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux

In [3]:
#no
!pip install typing-extensions

In [2]:
import collections.abc
import sys
import torch
from transformers import AutoModelForCausalLM

# Temporarily fix the ImportError for collections.Mapping
sys.modules['collections.Mapping'] = collections.abc.Mapping
sys.modules['collections.MutableMapping'] = collections.abc.MutableMapping
sys.modules['collections.Sequence'] = collections.abc.Sequence

# Now try importing the deepseek_vl package
from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM
from deepseek_vl.utils.io import load_pil_images

Python version is above 3.10, patching the collections module.


In [3]:
import cv2
from PIL import Image
import tensorflow as tf

In [4]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU' or x.device_type == 'CPU']

print(get_available_devices())



['/device:CPU:0', '/device:GPU:0']


In [5]:
with tf.device('/device:GPU:0'):
  # Initialize the model
  model_path = "deepseek-ai/deepseek-vl-1.3b-base"
  vl_chat_processor = VLChatProcessor.from_pretrained(model_path)
  tokenizer = vl_chat_processor.tokenizer

  # Load the model
  vl_gpt = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
  vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


processor_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

In [6]:
# Function to capture a frame from the webcam
captured_frame = None
def capture_frame(video_input):
    global captured_frame
    ret, frame = video_input.read()
    if ret:
        captured_frame = frame
        cv2.imwrite('captured_frame.png', frame)
        return frame
    return None

In [7]:
# Function to run inference on the captured frame
def run_inference():
    if captured_frame is None:
        return "No frame captured. Please press the 'Run' button to capture a frame."

    image_path = 'captured_frame.png'
    conversation = [
        {
            "role": "User",
            "content": "<image_placeholder>Describe this image.",
            "images": [image_path]
        },
        {
            "role": "Assistant",
            "content": ""
        }
    ]

    # Load the image and prepare for inputs
    pil_images = load_pil_images(conversation)
    prepare_inputs = vl_chat_processor(
        conversations=conversation,
        images=pil_images,
        force_batchify=True
    ).to(vl_gpt.device)

    # Run the image encoder to get the image embeddings
    inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

    # Generate the response from the model
    outputs = vl_gpt.language_model.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=prepare_inputs.attention_mask,
        pad_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        do_sample=False,
        use_cache=True
    )

    # Decode and return the response
    answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
    return f"{prepare_inputs['sft_format'][0]}: {answer}"


In [8]:
# Function to process user question about the captured frame
def ask_question(question):
    if captured_frame is None:
        return "No frame captured. Please press the 'Run' button to capture a frame."

    image_path = 'captured_frame.png'
    conversation = [
        {
            "role": "User",
            "content": f"<image_placeholder>{question}",
            "images": [image_path]
        },
        {
            "role": "Assistant",
            "content": ""
        }
    ]

    # Load the image and prepare for inputs
    pil_images = load_pil_images(conversation)
    prepare_inputs = vl_chat_processor(
        conversations=conversation,
        images=pil_images,
        force_batchify=True
    ).to(vl_gpt.device)

    # Run the image encoder to get the image embeddings
    inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

    # Generate the response from the model
    outputs = vl_gpt.language_model.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=prepare_inputs.attention_mask,
        pad_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        do_sample=False,
        use_cache=True
    )

    # Decode and return the response
    answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
    return f"Q: {question}\nA: {answer}"


In [11]:
# Gradio interface setup
import gradio as gr
def video_feed():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield frame

video_component = gr.Video(video_feed, live=True)
run_button = gr.Button("Run")
chatbox = gr.Chatbox()

def on_run():
    video_input = cv2.VideoCapture(0)
    capture_frame(video_input)
    video_input.release()
    return "Frame captured. You can now ask questions about the image."

run_button.click(on_run)

def on_ask_question(question):
    return ask_question(question)

chatbox.submit(on_ask_question)

gr.Interface(
    fn=None,
    inputs=[],
    outputs=[video_component, run_button, chatbox],
    live=True
).launch(share=True)

TypeError: Video.__init__() got an unexpected keyword argument 'live'